In [11]:
import pandas as pd
import numpy as np
from tableschema import infer
from modules.utils.utils import read_json , initialize_tables_in_db, load_to_postgres, build_connection_engine
from pathlib import Path

p = Path(".")

config = read_json(p / "modules"/"config.json")
table_schema = 'remote'


# 1 - Load raw data

In [12]:
sheets = pd.ExcelFile('Data Analyst Assignment.xlsx')

In [13]:
# here we load the data from the sheets into pandas DF
# this will generate a dictionary that maps all the sheets
dct = pd.read_excel(sheets, sheet_name=None)
        

# 2 - Load raw data in staging tables in Postgres DB

Here I want to 
1. Load all the raw data into postgres into the public.table schema. This will be our raw/stagin section
2. Once all the data is loaded, I can perform the EDA and evaluate how to normalize the table to account for missing data, bad data

In [14]:
for key in dct:
    print(len(dct[key]))

99
18484
569
11
3
39409
60855


In [15]:
initialize_tables_in_db(config, dct, table_schema)

dimaccounts
False
Initializing table
dimcustomer
False
Initializing table
dimproduct
False
Initializing table
dimsalesterritory
False
Initializing table
dimscenario
False
Initializing table
factfinance
False
Initializing table
factresellersales
False
Initializing table


In [16]:
load_to_postgres(config, dct, table_schema)

<class 'str'>
dimaccounts
Inserted 99 rows in remote.dimaccounts
<class 'str'>
dimcustomer
Inserted 18484 rows in remote.dimcustomer
<class 'str'>
dimproduct
Inserted 569 rows in remote.dimproduct
<class 'str'>
dimsalesterritory
Inserted 11 rows in remote.dimsalesterritory
<class 'str'>
dimscenario
Inserted 3 rows in remote.dimscenario
<class 'str'>
factfinance
Inserted 39409 rows in remote.factfinance
<class 'str'>
factresellersales
Inserted 60855 rows in remote.factresellersales


# 3 - Perform EDA on the raw data

### EDA on DimAccounts

In [17]:
p = Path(".")

config = read_json(p / "modules"/"config.json")
conn_s = build_connection_engine(config, 's')

In [22]:
df = pd.read_sql_query(f"select * from {table_schema}.dimcustomer ",con=conn_s)

# issues 

- dimaccounts -> potentially faulty custommembers 
- dimcustomer -> birthdate, datefirstpurchase, phone number (strip)

In [23]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18484 entries, 0 to 18483
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   customerkey           18484 non-null  float64
 1   geographykey          18484 non-null  float64
 2   customeralternatekey  18484 non-null  object 
 3   title                 101 non-null    object 
 4   firstname             18484 non-null  object 
 5   middlename            10654 non-null  object 
 6   lastname              18484 non-null  object 
 7   namestyle             18484 non-null  float64
 8   birthdate             18484 non-null  object 
 9   maritalstatus         18484 non-null  object 
 10  suffix                3 non-null      object 
 11  gender                18484 non-null  object 
 12  emailaddress          18484 non-null  object 
 13  yearlyincome          18484 non-null  float64
 14  totalchildren         18484 non-null  float64
 15  numberchildrenathom

In [24]:
df

,customerkey,geographykey,customeralternatekey,title,firstname,middlename,lastname,namestyle,birthdate,maritalstatus,...,englishoccupation,spanishoccupation,frenchoccupation,houseownerflag,numbercarsowned,addressline1,addressline2,phone,datefirstpurchase,commutedistance
0,11000.0,26.0,AW00011000,None,Jon,V,Yang,0.0,1971-06-10 00:00:00,M,...,Professional,Profesional,Cadre,1.0,0.0,3761 N. 14th St,None,1 (11) 500 555-0162,19/01/11,1-2 Miles
1,11001.0,37.0,AW00011001,None,Eugene,L,Huang,0.0,1976-10-05 00:00:00,S,...,Professional,Profesional,Cadre,0.0,1.0,2243 W St.,None,1 (11) 500 555-0110,15/01/11,0-1 Miles
2,11002.0,31.0,AW00011002,None,Ruben,None,Torres,0.0,1971-09-02 00:00:00,M,...,Professional,Profesional,Cadre,1.0,1.0,5844 Linden Land,None,1 (11) 500 555-0184,2011-07-01 00:00:00,2-5 Miles
3,11003.0,11.0,AW00011003,None,Christy,None,Zhu,0.0,14/08/73,S,...,Professional,Profesional,Cadre,0.0,1.0,1825 Village Pl.,None,1 (11) 500 555-0162,29/12/10,5-10 Miles
4,11004.0,19.0,AW00011004,None,Elizabeth,None,Johnson,0.0,1979-05-08 00:00:00,S,...,Professional,Profesional,Cadre,1.0,4.0,7553 Harness Circle,None,1 (11) 500 555-0131,23/01/11,1-2 Miles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18479,29479.0,209.0,AW00029479,None,Tommy,L,Tang,0.0,30/06/69,M,...,Clerical,Administrativo,Employé,1.0,0.0,"111, rue Maillard",None,1 (11) 500 555-0136,2012-04-09 00:00:00,0-1 Miles
18480,29480.0,248.0,AW00029480,None,Nina,W,Raji,0.0,1977-06-05 00:00:00,S,...,Clerical,Administrativo,Employé,1.0,0.0,9 Katherine Drive,None,1 (11) 500 555-0146,17/07/13,0-1 Miles
18481,29481.0,120.0,AW00029481,None,Ivan,None,Suri,0.0,1965-04-07 00:00:00,S,...,Clerical,Administrativo,Employé,0.0,0.0,Knaackstr 4,None,1 (11) 500 555-0144,13/08/11,0-1 Miles
18482,29482.0,179.0,AW00029482,None,Clayton,None,Zhang,0.0,1964-01-09 00:00:00,M,...,Clerical,Administrativo,Employé,1.0,0.0,"1080, quai de Grenelle",None,1 (11) 500 555-0137,18/09/12,0-1 Miles
